In [ ]:
# hide
!nvidia-smi

Mon May 31 15:53:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# hide
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq fastai transformers datasets wandb rouge-score nltk
    !pip install -q git+git://github.com/aikindergarten/fasthugs.git

     |████████████████████████████████| 204kB 25.1MB/s 
     |████████████████████████████████| 2.3MB 46.2MB/s 
     |████████████████████████████████| 235kB 55.5MB/s 
     |████████████████████████████████| 1.8MB 39.5MB/s 
     |████████████████████████████████| 1.5MB 47.1MB/s 
     |████████████████████████████████| 61kB 10.7MB/s 
     |████████████████████████████████| 901kB 42.4MB/s 
     |████████████████████████████████| 3.3MB 35.5MB/s 
     |████████████████████████████████| 112kB 52.6MB/s 
     |████████████████████████████████| 245kB 58.9MB/s 
     |████████████████████████████████| 174kB 49.9MB/s 
     |████████████████████████████████| 102kB 15.3MB/s 
     |████████████████████████████████| 133kB 56.7MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 


In [ ]:
#all_slow

# Abstractive summarization with T5

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, MarianConfig, AutoTokenizer, AutoConfig
from datasets import load_dataset, load_metric, concatenate_datasets
from fastai.text.all import *
from fastai.callback.wandb import *

from fasthugs.learner import TransLearner
from fasthugs.data import TransformersTextBlock, TextGetter, get_splits

## Setup

Let's define main settings for the run in one place:

In [ ]:
model_name = "t5-small"

max_len = 512
bs = 8
val_bs = bs*2

lr = 2e-5

In [ ]:
#hide_output
datasets = load_dataset("xsum")
metric = load_metric("rouge")

Using custom data configuration default


Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499. Subsequent calls will reuse this data.


## Dataloaders

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# splits = get_splits(datasets)
# train_ds = concatenate_datasets([datasets['train'], datasets['validation']])

In [ ]:
train_ds = datasets['validation']
splits = RandomSplitter()(train_ds)

In [ ]:
@ItemTransform
def untuple1(x):
    return (*x[0], )

In [ ]:
dblock = DataBlock(
    blocks = [TransformersTextBlock(tokenizer=tokenizer, do_targets=True, with_labels=True)],
    get_x=TextGetter('document', 'summary', prefix1='summarize: '),
    item_tfms=untuple1,
    splitter=IndexSplitter(splits[1]))

In [ ]:
#hide
# dblock.summary(df)

In [ ]:
%%time
bs = 16
dls = dblock.dataloaders(train_ds, bs=bs, val_bs=bs*2, shuffle=True)

CPU times: user 3.13 s, sys: 37.5 ms, total: 3.16 s
Wall time: 3.14 s


In [ ]:
#hide
b = dls.one_batch()
b

({'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          ...,
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0'),
  'input_ids': tensor([[21603,    10,    37,  ...,   112, 23761,     1],
          [21603,    10,    37,  ...,  5054,     3,     1],
          [21603,    10,  9145,  ...,     8,  2968,     1],
          ...,
          [21603,    10, 10582,  ...,    34,    91,     1],
          [21603,    10,  1186,  ...,     8,  3202,     1],
          [21603,    10,    37,  ...,   739,   179,     1]], device='cuda:0'),
  'labels': tensor([[  282,   373,  1330,    12,    36,     8,   495,     6,     8,  6971,
              15,   888,    13,     8,  1248,  2025,  2034,    47,   646,    12,
               8,   804,   239,     6,    28,     3, 21031,   122,   173,  4049,
             309, 25548,    31,     7,   307,    18,   235,  2810,    26

In [ ]:
#collapse
dls.show_batch(max_n=4)

,text,text_
0,"summarize: The £13m fee, along with the £2m paid by Sunderland for fellow defender Adam Matthews, suggests that the Scottish champions once again emerged from summer dealings having made a profit. Striker Nadir Ciftci, from Dundee United, and defender Dedryck Boyata, from Manchester City, arrived for about £1.5m each, while Celtic still had some spare change to purchase centre-back Jozo Simunovic from Dinamo Zagreb and midfielder Ryan Christie from Inverness Caledonian Thistle on the final day of the window. The number of other clubs willing or able to pay transfer or compensation fees has been on the increase and Dundee, Dundee United, Hearts, Motherwell and Ross County have all loosened the purse strings to some degree. However, cost cutting remains as squad sizes continue to shrink County under Jim McIntyre continued where Derek Adams left off in previous seasons with the biggest turnover of staff along with Motherwell, where an","As always seems to be the case, the marquee move of the summer transfer window was left to the final day, with Virgil van Dijk's long-touted switch to Southampton from Celtic finally being completed."
1,"summarize: Manchester City and Liverpool played out a pulsating 1-1 draw and Manchester United moved into fifth as a 3-1 win at Middlesbrough saw them move above Arsenal, who lost 3-1 at West Brom. Elsewhere, Bournemouth steered well clear of the drop thanks to a 2-0 victory over Swansea, while there were wins for Everton, Crystal Palace and Leicester City. Pick your XI from our list and share with your friends. Do you agree with my team of the week or would you go for a different team? Why not pick your very own team of the week from the shortlist selected by BBC Sport journalists and share it with your friends? Two of the best saves you will see from Kasper Schmeichel, and reminiscent of his father Peter Schmeichel at his best. The save from Andy Carroll's header was immense and the shot stop from the same player was","Chelsea maintained their 10-point lead at the top of the Premier League with a 2-1 win at Stoke, while Tottenham beat Southampton 2-1 at home to stay second."
2,"summarize: I'm scared because there is also a clock counting down in the bottom right-hand corner so something important must be about to happen like the arrival of the four horsemen of the Apocalypse but it turns out Shaun Batt has joined Leyton Orient from Millwall instead. On loan. Welcome to Transfer Deadline Day, which I've deliberately written in capitals because there are those who think we should make it a public holiday. Chaotic and stressful and full of panic buying, Transfer Deadline Day and the window that precedes it make up football's equivalent of the Yuletide season, with Sky Sports' Jim White playing its equivalent of Santa. ""Sorry, no joy with Fabregas, I brought you Fellaini instead."" A magnificent irony of transfer windows is that they were introduced in 2002-03 in part to stabilise the game. It's about ducking and diving, maybe finding a player whose mother-in-law lives in",There is a man on my telly standing outside The Den in a suit speaking fast and portentously without full stops or commas and with lots of superlatives.
3,"summarize: Rivals were awestruck by the amount of mileage the team were able to manage with a brand new car. Mercedes were determined to turn so many laps that they had to start splitting each day between drivers Lewis Hamilton and Nico Rosberg to avoid over-exerting them. ""Amazing,"" said Williams technical chief Pat Symonds. Another senior figure put it into context: ""800km is an entire grand prix weekend in one day."" Not everyone was charitable, though. ""It looks like they're gearing up to take that car to Le Mans,"" said one rival. ""It's admirable, but it's taking the Mickey a bit. You don't need to do that many miles."" Perhaps not. But, at this stage of the preparations, as Hamilton put it: 'Mileage - that's the key. To try and get as much mileage as 

## Tracking

In [ ]:
import wandb

WANDB_NAME = f'{ds_name}-{model_name}'
GROUP = f'{ds_name}-{model_name}-simple-{lr:.0e}'
NOTES = f'finetuning {model_name} with RAdam lr={lr:.0e}'
CONFIG = {}
TAGS =[model_name, ds_name, 'radam']

In [ ]:
#hide_output
wandb.init(reinit=True, project="fasthugs", entity="fastai_community",
           name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG);

## Training

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
learn = TransLearner(dls, model, loss_func=noop)

In [ ]:
learn.validate()

(#1) [3.882331132888794]

In [ ]:
learn.fit_one_cycle(2, 1e-4)

epoch,train_loss,valid_loss,time
0,2.835807,2.572247,05:25
1,2.720766,2.543403,05:33


In [ ]:
learn.validate()

(#1) [2.543402671813965]

In [ ]:
sample = train_ds[0]
print(f"document: {sample['document']}")
print(f"\nreference summary: {sample['summary']}")
inp = tokenizer('summarize: '+sample['document'], return_tensors='pt')
pred = learn.generate(inp['input_ids'].to(dls.device))
out = tokenizer.decode(pred[0].cpu(), skip_special_tokens=True)
print(f"\npredicted summary: {out}")

document: The country's consumer watchdog has taken Apple to court for false advertising because the tablet computer does not work on Australia's 4G network.
Apple's lawyers said they were willing to publish a clarification.
However the company does not accept that it misled customers.
The Australian Competition and Consumer Commission (ACCC) said on Tuesday: "Apple's recent promotion of the new 'iPad with wi-fi + 4G' is misleading because it represents to Australian consumers that the product can, with a sim card, connect to a 4G mobile data network in Australia, when this is not the case."
The watchdog then lodged a complaint at the Federal Court in Melbourne.
At a preliminary hearing, Apple lawyer Paul Anastassiou said Apple had never claimed the device would work fully on the current 4G network operated by Telstra.
Apple says the new iPad works on what is globally accepted to be a 4G network.
The matter will go to a full trial on 2 May.
The Apple iPad's third version went on sale e